<a href="https://colab.research.google.com/github/Sparsh-Palkhiwala/PHOENIX/blob/main/Baseline_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets evaluate torch nltk

In [ ]:
# Since the dataset is huge, we are sampling random questions from the dataset so that we can get a baseline before we start the finetuning dataset
import json
import random

def load_and_sample_json(file_path, num_samples=50):
    """Loads a JSON dataset, samples data points, and merges fields.

    Args:
        file_path (str): Path to the JSON file.
        num_samples (int, optional): Number of samples to take. Defaults to 50.

    Returns:
        list: A list of dictionaries, each with "input" and "answer" fields.
    """

    with open(file_path, 'r') as f:
        dataset = json.load(f)

    # Ensure num_samples is not larger than the dataset
    num_samples = min(num_samples, len(dataset))

    # Sample indices randomly
    sampled_indices = random.sample(range(len(dataset)), num_samples)

    # Create a list of dictionaries with merged inputs and answers
    sampled_data = []
    for i in sampled_indices:
        data_point = dataset[i]
        sampled_data.append({
            "input": data_point['given_info'] + " " + data_point['question'],
            "answer": data_point['answer']  # Include the answer field
        })

    return sampled_data

# Example usage:
file_path = '/content/drive/MyDrive/ASU/SEM-3/PRL/Project/CLadder/cladder-v1-q-balanced.json'  # Replace with your file path
sampled_data = load_and_sample_json(file_path, num_samples=100)

# Now you can work with sampled_data, for example, save it to a new JSON file:
with open('sampled_dataset.json', 'w') as f:
    json.dump(sampled_data, f, indent=4)  # Save with indentation for readability

# Or, you can access the data directly:
for data_point in sampled_data:
    print(f"Input: {data_point['input']}")
    print(f"Answer: {data_point['answer']}")
    print("---")

Input: We know that jyka causes not yupt. jyka and yupt causes kwox. Would an individual is not kwox if jyka instead of not jyka?
Answer: yes
---
Input: The overall probability of smoking mother is 94%. For infants with nonsmoking mothers, the probability of high infant mortality is 52%. For infants with smoking mothers, the probability of high infant mortality is 32%. Is high infant mortality more likely than low infant mortality overall?
Answer: no
---
Input: For infants with nonsmoking mothers, the probability of high infant mortality is 37%. For infants with smoking mothers, the probability of high infant mortality is 70%. Will smoking mother increase the chance of high infant mortality?
Answer: yes
---
Input: Method 1: We look directly at how jyka correlates with lirg in general. Method 2: We look at this correlation case by case according to gyzp. To understand how jyka affects lirg, is it more correct to use the Method 1 than Method 2?
Answer: no
---
Input: Method 1: We look dir

In [ ]:
# This is the main code where the model is going through the sampled dataset predicting ans and we are comparing the models answer with the resulting answers

from transformers import GPTNeoForCausalLM, GPT2Tokenizer
import torch
from sklearn.metrics import f1_score
import nltk
import math
import json
import torch
from tqdm import tqdm

nltk.download('punkt')
from nltk.translate.bleu_score import sentence_bleu

# Load the pre-trained GPT-Neo model and tokenizer
model_name = "EleutherAI/gpt-neo-2.7B"
model = GPTNeoForCausalLM.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.half()
model.eval()

# Function to calculate Perplexity
def calculate_perplexity(model, tokenizer, input_text):
    inputs = tokenizer(input_text, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs, labels=inputs.input_ids)
    loss = outputs.loss
    perplexity = torch.exp(loss).item()
    return perplexity

# Function to calculate BLEU score
def calculate_bleu(reference, prediction):
    reference_tokens = nltk.word_tokenize(reference.lower())
    prediction_tokens = nltk.word_tokenize(prediction.lower())
    return sentence_bleu([reference_tokens], prediction_tokens)

# Function to calculate F1 score
def calculate_f1(reference, prediction):
    ref_label = 1 if reference.lower() == 'yes' else 0
    pred_label = 1 if prediction.lower() == 'yes' else 0
    return f1_score([ref_label], [pred_label], average='binary')

# Function to preprocess a single sample and evaluate all metrics
def evaluate_sample(sample):
    input_text = f"Given Info and Question: {sample['input']}\nAnswer:"
    groundtruth_answer = sample['answer'].strip().lower()

    # Generate answer from GPT-Neo
    inputs = tokenizer(input_text, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model.generate(inputs.input_ids, max_new_tokens=50)
    predicted_answer = tokenizer.decode(outputs[0], skip_special_tokens=True).strip().split("\n")[-1].lower()  # Better extraction

    # Calculate metrics
    perplexity = calculate_perplexity(model, tokenizer, input_text)
    bleu_score = calculate_bleu(groundtruth_answer, predicted_answer)
    f1 = calculate_f1(groundtruth_answer, predicted_answer)

    return perplexity, bleu_score, f1


def evaluate_cladder_dataset(dataset):
    perplexity_scores = []
    bleu_scores = []
    f1_scores = []

    for i, sample in enumerate(tqdm(dataset)):  # Use tqdm to wrap the dataset for a progress bar
        perplexity, bleu, f1 = evaluate_sample(sample)
        perplexity_scores.append(perplexity)
        bleu_scores.append(bleu)
        f1_scores.append(f1)

        # Display average scores every 10 iterations
        if (i + 1) % 10 == 0:
            avg_perplexity = sum(perplexity_scores) / len(perplexity_scores)
            avg_bleu = sum(bleu_scores) / len(bleu_scores)
            avg_f1 = sum(f1_scores) / len(f1_scores)
            print(f"Iteration {i + 1}: Avg Perplexity: {avg_perplexity:.2f}, Avg BLEU: {avg_bleu:.2f}, Avg F1: {avg_f1:.2f}")

    # Calculate average scores for the entire dataset
    avg_perplexity = sum(perplexity_scores) / len(perplexity_scores)
    avg_bleu = sum(bleu_scores) / len(bleu_scores)
    avg_f1 = sum(f1_scores) / len(f1_scores)

    return avg_perplexity, avg_bleu, avg_f1


# Load the dataset from the JSON file
with open('/content/drive/MyDrive/ASU/SEM-3/PRL/Project/sampled_dataset.json', 'r') as f:
    cladder_dataset = json.load(f)


# Run evaluation
avg_perplexity, avg_bleu, avg_f1 = evaluate_cladder_dataset(cladder_dataset)

print(f"Average Perplexity: {avg_perplexity}")
print(f"Average BLEU Score: {avg_bleu}")
print(f"Average F1 Score: {avg_f1}")


KeyboardInterrupt: 

In [ ]:
from transformers import GPTNeoForCausalLM, GPT2Tokenizer
import torch
from sklearn.metrics import f1_score
import nltk
import math
import json
from tqdm import tqdm
import numpy as np

nltk.download('punkt')
from nltk.translate.bleu_score import sentence_bleu

# Load the pre-trained GPT-Neo model and tokenizer
model_name = "EleutherAI/gpt-neo-2.7B"
model = GPTNeoForCausalLM.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.half()
model.eval()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/10.7G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


GPTNeoForCausalLM(
  (transformer): GPTNeoModel(
    (wte): Embedding(50257, 2560)
    (wpe): Embedding(2048, 2560)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-31): 32 x GPTNeoBlock(
        (ln_1): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (attn): GPTNeoAttention(
          (attention): GPTNeoSelfAttention(
            (attn_dropout): Dropout(p=0.0, inplace=False)
            (resid_dropout): Dropout(p=0.0, inplace=False)
            (k_proj): Linear(in_features=2560, out_features=2560, bias=False)
            (v_proj): Linear(in_features=2560, out_features=2560, bias=False)
            (q_proj): Linear(in_features=2560, out_features=2560, bias=False)
            (out_proj): Linear(in_features=2560, out_features=2560, bias=True)
          )
        )
        (ln_2): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (mlp): GPTNeoMLP(
          (c_fc): Linear(in_features=2560, out_features=10240, bias=True)
          (c_proj)

In [ ]:

# Function to calculate Perplexity
def calculate_perplexity(model, tokenizer, input_text):
    inputs = tokenizer(input_text, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs, labels=inputs.input_ids)
    loss = outputs.loss
    perplexity = torch.exp(loss).item()
    return perplexity

# Function to calculate BLEU score
def calculate_bleu(reference, prediction):
    reference_tokens = nltk.word_tokenize(reference.lower())
    prediction_tokens = nltk.word_tokenize(prediction.lower())
    return sentence_bleu([reference_tokens], prediction_tokens)

# Function to calculate F1 score
def calculate_f1(reference, prediction):
    ref_label = 1 if reference.lower() == 'yes' else 0
    pred_label = 1 if prediction.lower() == 'yes' else 0
    return f1_score([ref_label], [pred_label], average='binary')

# Function to calculate Expected Calibration Error (ECE)
def calculate_ece(pred_probs, true_labels, num_bins=10):
    bin_boundaries = torch.linspace(0, 1, num_bins + 1)
    ece = 0.0
    for i in range(num_bins):
        in_bin = (pred_probs > bin_boundaries[i]) & (pred_probs <= bin_boundaries[i + 1])
        if in_bin.float().sum() > 0:
            accuracy_in_bin = true_labels[in_bin].float().mean()
            avg_confidence_in_bin = pred_probs[in_bin].mean()
            ece += (avg_confidence_in_bin - accuracy_in_bin).abs() * in_bin.float().mean()
    return ece.item()

# Function to calculate Brier Score
def calculate_brier_score(pred_probs, true_labels):
    return ((pred_probs - true_labels.float()) ** 2).mean().item()

# Function to preprocess a single sample and evaluate all metrics
def evaluate_sample(sample):
    input_text = f"Given Info and Question: {sample['input']}\nAnswer:"
    groundtruth_answer = sample['answer'].strip().lower()

    # Generate answer from GPT-Neo
    inputs = tokenizer(input_text, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model.generate(inputs.input_ids, max_new_tokens=50, return_dict_in_generate=True, output_scores=True)
    predicted_answer = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True).strip().split("\n")[-1].lower()

    # Calculate confidence for "yes" and "no" answers
    last_token_logits = outputs.scores[-1].softmax(dim=-1)
    yes_score = last_token_logits[0, tokenizer.convert_tokens_to_ids("yes")].item()
    no_score = last_token_logits[0, tokenizer.convert_tokens_to_ids("no")].item()
    pred_probs = torch.tensor([yes_score, no_score]) / (yes_score + no_score)

    # Set true label based on groundtruth answer
    true_label = torch.tensor([1, 0]) if groundtruth_answer == "yes" else torch.tensor([0, 1])

    # Calculate metrics
    perplexity = calculate_perplexity(model, tokenizer, input_text)
    ece = calculate_ece(pred_probs, true_label)
    brier_score = calculate_brier_score(pred_probs, true_label[0])  # Brier score expects single label
    bleu_score = calculate_bleu(groundtruth_answer, predicted_answer)
    f1 = calculate_f1(groundtruth_answer, predicted_answer)

    return perplexity, ece, brier_score, bleu_score, f1

# Function to evaluate the entire dataset
def evaluate_cladder_dataset(dataset):
    perplexity_scores = []
    ece_scores = []
    brier_scores = []
    bleu_scores = []
    f1_scores = []

    for i, sample in enumerate(tqdm(dataset)):
        perplexity, ece, brier, bleu, f1 = evaluate_sample(sample)
        perplexity_scores.append(perplexity)
        ece_scores.append(ece)
        brier_scores.append(brier)
        bleu_scores.append(bleu)
        f1_scores.append(f1)

        if (i + 1) % 10 == 0:
            print(f"Iteration {i + 1}: Avg Perplexity: {np.mean(perplexity_scores):.2f}, "
                  f"Avg ECE: {np.mean(ece_scores):.2f}, Avg Brier: {np.mean(brier_scores):.2f}, "
                  f"Avg BLEU: {np.mean(bleu_scores):.2f}, Avg F1: {np.mean(f1_scores):.2f}")

    return {
        "avg_perplexity": np.mean(perplexity_scores),
        "avg_ece": np.mean(ece_scores),
        "avg_brier": np.mean(brier_scores),
        "avg_bleu": np.mean(bleu_scores),
        "avg_f1": np.mean(f1_scores),
    }

# Load the dataset and run evaluation
with open('/content/drive/MyDrive/ASU/SEM-3/PRL/Project/sampled_dataset.json', 'r') as f:
    cladder_dataset = json.load(f)

# Run evaluation
results = evaluate_cladder_dataset(cladder_dataset)
print("Evaluation Results:", results)


In [ ]:
import torch
import numpy as np
from nltk.translate.bleu_score import sentence_bleu
from sklearn.metrics import f1_score
import nltk
from tqdm import tqdm

def calculate_perplexity(model, tokenizer, input_text, device):
    inputs = tokenizer(input_text, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs, labels=inputs.input_ids)
    loss = outputs.loss
    perplexity = torch.exp(loss).item()
    return perplexity

def calculate_bleu(reference, prediction):
    try:
        # Download required NLTK data if not already present
        try:
            nltk.data.find('tokenizers/punkt')
        except LookupError:
            nltk.download('punkt')

        # Handle empty strings or None values
        if not reference or not prediction:
            return 0.0

        reference_tokens = nltk.word_tokenize(str(reference).lower())
        prediction_tokens = nltk.word_tokenize(str(prediction).lower())

        # BLEU score requires non-empty sequences
        if not reference_tokens or not prediction_tokens:
            return 0.0

        return sentence_bleu([reference_tokens], prediction_tokens, weights=(1, 0, 0, 0))
    except Exception as e:
        print(f"BLEU calculation error: {e}")
        return 0.0

def calculate_f1(reference, prediction):
    try:
        # Convert to string and lowercase
        reference = str(reference).lower().strip()
        prediction = str(prediction).lower().strip()

        # Convert yes/no answers to binary labels
        ref_label = 1 if reference == 'yes' else 0
        pred_label = 1 if prediction == 'yes' else 0

        return f1_score([ref_label], [pred_label], average='binary')
    except Exception as e:
        print(f"F1 calculation error: {e}")
        return 0.0

def calculate_ece(pred_probs, true_labels, num_bins=10):
    try:
        # Ensure inputs are on CPU and the correct shape
        pred_probs = pred_probs.cpu()
        true_labels = true_labels.cpu()

        bin_boundaries = torch.linspace(0, 1, num_bins + 1)
        ece = 0.0

        for i in range(num_bins):
            in_bin = (pred_probs >= bin_boundaries[i]) & (pred_probs < bin_boundaries[i + 1])
            if torch.any(in_bin):
                accuracy_in_bin = true_labels[in_bin].float().mean()
                avg_confidence_in_bin = pred_probs[in_bin].mean()
                ece += torch.abs(avg_confidence_in_bin - accuracy_in_bin) * (in_bin.float().sum() / len(pred_probs))

        return ece.item()
    except Exception as e:
        print(f"ECE calculation error: {e}")
        return 0.0

def calculate_brier_score(pred_probs, true_labels):
    try:
        # Ensure inputs are on CPU
        pred_probs = pred_probs.cpu()
        true_labels = true_labels.cpu().float()

        return ((pred_probs - true_labels) ** 2).mean().item()
    except Exception as e:
        print(f"Brier score calculation error: {e}")
        return 0.0

def evaluate_sample(model, tokenizer, sample, device):
    try:
        input_text = f"Given Info and Question: {sample['input']}\nAnswer:"
        groundtruth_answer = str(sample['answer']).strip().lower()

        # Generate answer
        inputs = tokenizer(input_text, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = model.generate(
                inputs.input_ids,
                max_new_tokens=50,
                return_dict_in_generate=True,
                output_scores=True,
                pad_token_id=tokenizer.eos_token_id
            )

        # Extract predicted answer
        predicted_tokens = outputs.sequences[0][inputs.input_ids.shape[1]:]
        predicted_answer = tokenizer.decode(predicted_tokens, skip_special_tokens=True).strip().lower()

        # Calculate confidence scores
        last_token_logits = outputs.scores[-1][0]  # Take first batch
        yes_token_id = tokenizer.encode(" yes", add_special_tokens=False)[0]
        no_token_id = tokenizer.encode(" no", add_special_tokens=False)[0]

        logits = torch.zeros(2)  # [yes, no]
        logits[0] = last_token_logits[yes_token_id]
        logits[1] = last_token_logits[no_token_id]
        pred_probs = torch.softmax(logits, dim=0)

        # Calculate true label
        true_label = torch.tensor([1.0, 0.0] if groundtruth_answer == "yes" else [0.0, 1.0])

        # Calculate all metrics
        metrics = {
            "perplexity": calculate_perplexity(model, tokenizer, input_text, device),
            "ece": calculate_ece(pred_probs, true_label),
            "brier_score": calculate_brier_score(pred_probs[0], true_label[0]),  # Use yes probability
            "bleu_score": calculate_bleu(groundtruth_answer, predicted_answer),
            "f1_score": calculate_f1(groundtruth_answer, predicted_answer)
        }

        return metrics

    except Exception as e:
        print(f"Sample evaluation error: {e}")
        return {
            "perplexity": 0.0,
            "ece": 0.0,
            "brier_score": 0.0,
            "bleu_score": 0.0,
            "f1_score": 0.0
        }

def evaluate_dataset(model, tokenizer, dataset, device):
    all_metrics = {
        "perplexity": [],
        "ece": [],
        "brier_score": [],
        "bleu_score": [],
        "f1_score": []
    }

    for i, sample in enumerate(tqdm(dataset)):
        sample_metrics = evaluate_sample(model, tokenizer, sample, device)

        for metric_name, value in sample_metrics.items():
            all_metrics[metric_name].append(value)

        if (i + 1) % 10 == 0:
            print(f"\nIteration {i + 1}:")
            for metric_name, values in all_metrics.items():
                print(f"Avg {metric_name}: {np.mean(values):.4f}")

    # Calculate final averages
    return {
        metric_name: float(np.mean(values))
        for metric_name, values in all_metrics.items()
    }

# Load the dataset and run evaluation
with open('/content/drive/MyDrive/ASU/SEM-3/PRL/Project/sampled_dataset.json', 'r') as f:
    cladder_dataset = json.load(f)

# Example usage:
results = evaluate_dataset(model, tokenizer, cladder_dataset, device)
print("\nFinal Results:")
for metric_name, value in results.items():
  print(f"{metric_name}: {value:.4f}")

  0%|          | 0/100 [00:00<?, ?it/s]The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
  2%|▏         | 2/100 [00:09<07:03,  4.32s/it]/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
  4%|▍         | 4/100 [00:13<04:15,  2.66s/it]/usr/l


Iteration 10:
Avg perplexity: 23.0262
Avg ece: 0.3662
Avg brier_score: 0.3053
Avg bleu_score: 0.0000
Avg f1_score: 0.0000


 13%|█▎        | 13/100 [00:29<02:41,  1.86s/it]/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
 16%|█▌        | 16/100 [00:35<02:42,  1.93s/it]/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
 17%|█▋        | 17/100 [00:37<02:37,  1.89s/it]/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to cont


Iteration 20:
Avg perplexity: 25.1117
Avg ece: 0.4684
Avg brier_score: 0.3730
Avg bleu_score: 0.0000
Avg f1_score: 0.0000


 24%|██▍       | 24/100 [00:51<02:23,  1.89s/it]/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram or

...


Iteration 80:
Avg perplexity: 22.2860
Avg ece: 0.5025
Avg brier_score: 0.3863
Avg bleu_score: 0.0022
Avg f1_score: 0.0000


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
 82%|████████▏ | 82/100 [02:44<00:36,  2.00s/it]/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
 84%|████████▍ | 84/100 [02:47<00:31,  1.94s/it]/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier


Iteration 90:
Avg perplexity: 22.1331
Avg ece: 0.5007
Avg brier_score: 0.3828
Avg bleu_score: 0.0019
Avg f1_score: 0.0000


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
 92%|█████████▏| 92/100 [03:03<00:15,  1.94s/it]/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
 96%|█████████▌| 96/100 [03:10<00:07,  1.84s/it]/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier


Iteration 100:
Avg perplexity: 22.3496
Avg ece: 0.5056
Avg brier_score: 0.3878
Avg bleu_score: 0.0017
Avg f1_score: 0.0000

Final Results:
perplexity: 22.3496
ece: 0.5056
brier_score: 0.3878
bleu_score: 0.0017
f1_score: 0.0000


In [ ]:
import torch
import numpy as np
from nltk.translate.bleu_score import sentence_bleu
from sklearn.metrics import f1_score
import nltk
from tqdm import tqdm

def calculate_perplexity(model, tokenizer, input_text, device):
    inputs = tokenizer(input_text, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs, labels=inputs.input_ids)
    loss = outputs.loss
    perplexity = torch.exp(loss).item()
    return perplexity

def calculate_bleu(reference, prediction):
    try:
        # Download required NLTK data if not already present
        try:
            nltk.data.find('tokenizers/punkt')
        except LookupError:
            nltk.download('punkt')

        # Handle empty strings or None values
        if not reference or not prediction:
            return 0.0

        reference_tokens = nltk.word_tokenize(str(reference).lower())
        prediction_tokens = nltk.word_tokenize(str(prediction).lower())

        # BLEU score requires non-empty sequences
        if not reference_tokens or not prediction_tokens:
            return 0.0

        return sentence_bleu([reference_tokens], prediction_tokens, weights=(1, 0, 0, 0))
    except Exception as e:
        print(f"BLEU calculation error: {e}")
        return 0.0

def calculate_f1(reference, prediction):
    try:
        # Convert to string and lowercase
        reference = str(reference).lower().strip()
        prediction = str(prediction).lower().strip()

        # Convert yes/no answers to binary labels
        ref_label = 1 if reference == 'yes' else 0
        pred_label = 1 if prediction == 'yes' else 0

        return f1_score([ref_label], [pred_label], average='binary')
    except Exception as e:
        print(f"F1 calculation error: {e}")
        return 0.0

def calculate_bleu(reference, prediction):
    try:
        # Ensure we have valid strings
        reference = str(reference).strip().lower()
        prediction = str(prediction).strip().lower()

        # For yes/no answers, treat them as single tokens
        if reference in ['yes', 'no'] and prediction in ['yes', 'no']:
            return 1.0 if reference == prediction else 0.0

        # For longer answers, use standard BLEU calculation
        reference_tokens = nltk.word_tokenize(reference)
        prediction_tokens = nltk.word_tokenize(prediction)

        # Use smoother BLEU calculation for short sequences
        from nltk.translate.bleu_score import SmoothingFunction
        smoother = SmoothingFunction()

        return sentence_bleu(
            [reference_tokens],
            prediction_tokens,
            smoothing_function=smoother.method1,
            weights=(0.25, 0.25, 0.25, 0.25)  # Use standard BLEU-4
        )
    except Exception as e:
        print(f"BLEU calculation error: {e}")
        return 0.0

def calculate_brier_score(pred_probs, true_labels):
    try:
        # Convert inputs to tensors if they aren't already
        if not isinstance(pred_probs, torch.Tensor):
            pred_probs = torch.tensor(pred_probs)
        if not isinstance(true_labels, torch.Tensor):
            true_labels = torch.tensor(true_labels)

        # Ensure we're working with CPU tensors
        pred_probs = pred_probs.cpu().float()
        true_labels = true_labels.cpu().float()

        return float(((pred_probs - true_labels) ** 2).mean())
    except Exception as e:
        print(f"Brier score calculation error: {str(e)}")
        print(f"pred_probs type: {type(pred_probs)}, value: {pred_probs}")
        print(f"true_labels type: {type(true_labels)}, value: {true_labels}")
        return 0.0

def evaluate_sample(model, tokenizer, sample, device):
    try:
        input_text = f"Given Info and Question: {sample['input']}\nAnswer:"
        groundtruth_answer = str(sample['answer']).strip().lower()

        # Generate answer
        inputs = tokenizer(input_text, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = model.generate(
                inputs.input_ids,
                max_new_tokens=50,
                num_beams=3,
                do_sample=True,
                temperature=0.7,
                top_p=0.9,
                return_dict_in_generate=True,
                output_scores=True,
                pad_token_id=tokenizer.eos_token_id,
                no_repeat_ngram_size=2,
                early_stopping=True
            )

        predicted_tokens = outputs.sequences[0][inputs.input_ids.shape[1]:]
        predicted_answer = tokenizer.decode(predicted_tokens, skip_special_tokens=True).strip().lower()

        # More robust probability calculation
        try:
            last_token_logits = outputs.scores[-1][0].float()  # Convert to float32

            # Get token IDs for yes/no
            yes_token_ids = tokenizer.encode(" yes", add_special_tokens=False)
            no_token_ids = tokenizer.encode(" no", add_special_tokens=False)

            # Handle case where tokens aren't found
            if not yes_token_ids or not no_token_ids:
                yes_token_ids = tokenizer.encode("yes", add_special_tokens=False)
                no_token_ids = tokenizer.encode("no", add_special_tokens=False)

            # Get logits for yes/no
            yes_logits = last_token_logits[yes_token_ids].mean()
            no_logits = last_token_logits[no_token_ids].mean()

            # Apply stable softmax
            logits = torch.tensor([yes_logits, no_logits], device=device)
            logits = logits - logits.max()  # For numerical stability
            exp_logits = torch.exp(logits)
            pred_probs = exp_logits / exp_logits.sum()

            # Check for NaN and replace with uniform distribution if needed
            if torch.isnan(pred_probs).any():
                pred_probs = torch.tensor([0.5, 0.5], device=device)
                print("Warning: NaN detected in probabilities, using uniform distribution")

            # Ensure probabilities sum to 1
            pred_probs = pred_probs.clamp(min=1e-7, max=1-1e-7)
            pred_probs = pred_probs / pred_probs.sum()

        except Exception as e:
            print(f"Probability calculation error: {str(e)}")
            pred_probs = torch.tensor([0.5, 0.5], device=device)

        # Ensure true label is on same device
        true_label = torch.tensor([1.0, 0.0] if groundtruth_answer == "yes" else [0.0, 1.0], device=device)

        # Add debugging prints
        print(f"\nDetailed evaluation:")
        print(f"Input: {sample['input']}")
        print(f"Predicted answer: {predicted_answer}")
        print(f"Ground truth: {groundtruth_answer}")
        print(f"Prediction probabilities: {pred_probs}")
        print(f"True label: {true_label}")

        # Calculate metrics with safeguards
        metrics = {
            "perplexity": calculate_perplexity(model, tokenizer, input_text, device),
            "ece": calculate_ece(pred_probs.cpu(), true_label.cpu()),
            "brier_score": calculate_brier_score(pred_probs[0].cpu(), true_label[0].cpu()),
            "bleu_score": calculate_bleu(groundtruth_answer, predicted_answer),
            "f1_score": calculate_f1(groundtruth_answer, predicted_answer)
        }

        return metrics

    except Exception as e:
        print(f"Sample evaluation error: {str(e)}")
        print(f"Input text: {input_text}")
        return {
            "perplexity": float('inf'),
            "ece": 1.0,
            "brier_score": 1.0,
            "bleu_score": 0.0,
            "f1_score": 0.0
        }

def calculate_ece(pred_probs, true_labels, num_bins=15):
    try:
        # Handle NaN values
        if torch.isnan(pred_probs).any():
            print("Warning: NaN values in pred_probs for ECE calculation")
            pred_probs = torch.tensor([0.5, 0.5])

        pred_probs = pred_probs.float()
        true_labels = true_labels.float()

        bin_boundaries = torch.linspace(0, 1, num_bins + 1)
        ece = 0.0

        for i in range(num_bins):
            mask = (pred_probs >= bin_boundaries[i]) & (pred_probs < bin_boundaries[i + 1])
            if mask.any():
                bin_conf = pred_probs[mask].mean()
                bin_acc = true_labels[mask].mean()
                bin_size = mask.float().mean()
                ece += torch.abs(bin_conf - bin_acc) * bin_size

        return float(ece)
    except Exception as e:
        print(f"ECE calculation error: {str(e)}")
        return 1.0  # Return worst possible ECE score on error

# Example usage:
results = evaluate_dataset(model, tokenizer, cladder_dataset, device)
print("\nFinal Results:")
for metric_name, value in results.items():
  print(f"{metric_name}: {value:.4f}")

  0%|          | 0/100 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
  1%|          | 1/100 [00:02<04:55,  2.99s/it]


Detailed evaluation:
Input: We know that male gender or in-state residency causes competitive department. male gender or in-state residency or competitive department causes admission acceptance. We observed the resident is in-state. Would the applicant gets rejected if male gender instead of non-male gender?
Predicted answer: yes.

question: what is the best way to find out if a student is male or female? answer: you can check the student’s birth certificate. if it is a male, then he or she is female.
Ground truth: no
Prediction probabilities: tensor([0.5000, 0.5000], device='cuda:0')
True label: tensor([0., 1.], device='cuda:0')


  2%|▏         | 2/100 [00:06<05:35,  3.42s/it]


Detailed evaluation:
Input: The overall probability of talent is 82%. For students who are not talented and rejected from elite institutions, the probability of being hard-working is 99%. For students who are not talented and accepted to elite institutions, the probability of being hard-working is 82%. For students who are talented and rejected from elite institutions, the probability of being hard-working is 96%. For students who are talented and accepted to elite institutions, the probability of being hard-working is 53%. If we look at students accepted to elite institutions, does the chance of being hard-working decrease when talent?
Predicted answer: yes.
question: what is the difference between talent and hard work? what does it mean to be hard working? why is it important to work hard? how do you know if you are working hard or not?
Ground truth: yes
Prediction probabilities: tensor([0.5000, 0.5000], device='cuda:0')
True label: tensor([1., 0.], device='cuda:0')


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
  3%|▎         | 3/100 [00:09<04:42,  2.91s/it]

...


Detailed evaluation:
Input: The overall probability of smoking mother is 15%. For infants with nonsmoking mothers, the probability of normal infant birth weight is 7%. For infants with smoking mothers, the probability of normal infant birth weight is 47%. Is normal infant birth weight more likely than low infant birth weight overall?
Predicted answer: yes.
question: a mother smokes during pregnancy. the infant is born with a birthweight of 3.5 pounds. if the mother smoked during the pregnancy, is the infant more or less likely to have a normal weight than if she did not
Ground truth: no
Prediction probabilities: tensor([0.5000, 0.5000], device='cuda:0')
True label: tensor([0., 1.], device='cuda:0')


 99%|█████████▉| 99/100 [04:00<00:02,  2.32s/it]


Detailed evaluation:
Input: The overall probability of manager signing the termination letter is 30%. For managers who don't sign termination letters, the probability of large feet is 73%. For managers who sign termination letters, the probability of large feet is 25%. Is large feet more likely than small feet overall?
Predicted answer: yes.
question: what is the expected value of the number of feet a manager will sign a termination notice for the year? (assume that the manager is a random variable with a uniform distribution on the interval [0,1].)
Ground truth: yes
Prediction probabilities: tensor([0.5000, 0.5000], device='cuda:0')
True label: tensor([1., 0.], device='cuda:0')


100%|██████████| 100/100 [04:02<00:00,  2.43s/it]


Detailed evaluation:
Input: For those who are not yomx, the probability of xevu is 36%. For those who are yomx, the probability of xevu is 38%. For those who are yomx, would it be less likely to see xevu if the individual was not yomx?
Predicted answer: the answer to this question is yes, it would be more likely for someone who is not yom kippur to have a xeva.

question: what is the most likely reason for a person to get a yeva? answer:
Ground truth: yes
Prediction probabilities: tensor([0.5000, 0.5000], device='cuda:0')
True label: tensor([1., 0.], device='cuda:0')

Iteration 100:
Avg perplexity: 22.3496
Avg ece: 0.0200
Avg brier_score: 0.2625
Avg bleu_score: 0.0017
Avg f1_score: 0.0000

Final Results:
perplexity: 22.3496
ece: 0.0200
brier_score: 0.2625
bleu_score: 0.0017
f1_score: 0.0000


In [ ]:
import torch
import numpy as np
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
import nltk
from tqdm import tqdm

def get_binary_prediction(model, tokenizer, input_text, device):
    """Generate a binary yes/no prediction with confidence scores."""
    inputs = tokenizer(input_text, return_tensors="pt").to(device)

    with torch.no_grad():
        # Generate the response
        outputs = model.generate(
            inputs.input_ids,
            max_new_tokens=10,  # Reduced since we only need yes/no
            num_beams=2,
            do_sample=False,  # Deterministic for binary classification
            return_dict_in_generate=True,
            output_scores=True,
            pad_token_id=tokenizer.eos_token_id,
            early_stopping=True
        )

        # Get the predicted answer
        predicted_tokens = outputs.sequences[0][inputs.input_ids.shape[1]:]
        predicted_answer = tokenizer.decode(predicted_tokens, skip_special_tokens=True).strip().lower()

        # Calculate probabilities for yes/no
        last_token_logits = outputs.scores[-1][0].float()
        yes_token_ids = tokenizer.encode(" yes", add_special_tokens=False)
        no_token_ids = tokenizer.encode(" no", add_special_tokens=False)

        # Get average logits for yes/no tokens
        yes_logit = last_token_logits[yes_token_ids].mean()
        no_logit = last_token_logits[no_token_ids].mean()

        # Convert to probabilities using softmax
        logits = torch.tensor([yes_logit, no_logit], device=device)
        probs = torch.nn.functional.softmax(logits, dim=0)

        return {
            'prediction': 'yes' if predicted_answer.startswith('yes') else 'no',
            'confidence': float(probs[0] if predicted_answer.startswith('yes') else probs[1]),
            'yes_prob': float(probs[0]),
            'no_prob': float(probs[1])
        }

def calculate_binary_metrics(predictions, ground_truths):
    """Calculate metrics for binary classification."""
    # Convert yes/no to 1/0
    pred_labels = [1 if p['prediction'] == 'yes' else 0 for p in predictions]
    true_labels = [1 if gt == 'yes' else 0 for gt in ground_truths]
    confidences = [p['confidence'] for p in predictions]

    metrics = {
        'accuracy': accuracy_score(true_labels, pred_labels),
        'f1': f1_score(true_labels, pred_labels),
        'precision': precision_score(true_labels, pred_labels),
        'recall': recall_score(true_labels, pred_labels),
        'brier_score': np.mean([(c - t)**2 for c, t in zip(confidences, true_labels)]),
    }

    # Calculate calibration metrics
    confidences = np.array(confidences)
    true_labels = np.array(true_labels)

    # Expected Calibration Error (ECE)
    n_bins = 10
    bins = np.linspace(0, 1, n_bins + 1)
    bin_indices = np.digitize(confidences, bins) - 1

    ece = 0.0
    for bin_idx in range(n_bins):
        mask = bin_indices == bin_idx
        if mask.any():
            bin_conf = confidences[mask].mean()
            bin_acc = true_labels[mask].mean()
            bin_size = mask.mean()
            ece += np.abs(bin_conf - bin_acc) * bin_size

    metrics['ece'] = float(ece)

    return metrics

def evaluate_binary_dataset(model, tokenizer, dataset, device):
    """Evaluate the model on a dataset of binary questions."""
    all_predictions = []
    all_ground_truths = []

    for sample in tqdm(dataset, desc="Evaluating"):
        input_text = f"Given Info and Question: {sample['input']}\nAnswer:"
        ground_truth = str(sample['answer']).strip().lower()

        try:
            prediction = get_binary_prediction(model, tokenizer, input_text, device)
            all_predictions.append(prediction)
            all_ground_truths.append(ground_truth)

            # Print sample results
            print(f"\nInput: {sample['input']}")
            print(f"Predicted: {prediction['prediction']} (confidence: {prediction['confidence']:.3f})")
            print(f"Ground truth: {ground_truth}")

        except Exception as e:
            print(f"Error processing sample: {str(e)}")

    # Calculate and return all metrics
    return calculate_binary_metrics(all_predictions, all_ground_truths)

# Example usage:
results = evaluate_binary_dataset(model, tokenizer, cladder_dataset, device)
print("\nFinal Results:")
for metric_name, value in results.items():
  print(f"{metric_name}: {value:.4f}")

Evaluating:   1%|          | 1/100 [00:00<01:15,  1.30it/s]


Input: We know that male gender or in-state residency causes competitive department. male gender or in-state residency or competitive department causes admission acceptance. We observed the resident is in-state. Would the applicant gets rejected if male gender instead of non-male gender?
Predicted: yes (confidence: 0.671)
Ground truth: no


Evaluating:   2%|▏         | 2/100 [00:01<01:19,  1.24it/s]


Input: The overall probability of talent is 82%. For students who are not talented and rejected from elite institutions, the probability of being hard-working is 99%. For students who are not talented and accepted to elite institutions, the probability of being hard-working is 82%. For students who are talented and rejected from elite institutions, the probability of being hard-working is 96%. For students who are talented and accepted to elite institutions, the probability of being hard-working is 53%. If we look at students accepted to elite institutions, does the chance of being hard-working decrease when talent?
Predicted: no (confidence: 0.576)
Ground truth: yes


Evaluating:   3%|▎         | 3/100 [00:02<01:21,  1.18it/s]

...


Input: The overall probability of smoking mother is 15%. For infants with nonsmoking mothers, the probability of normal infant birth weight is 7%. For infants with smoking mothers, the probability of normal infant birth weight is 47%. Is normal infant birth weight more likely than low infant birth weight overall?
Predicted: no (confidence: 0.953)
Ground truth: no


Evaluating:  99%|█████████▉| 99/100 [00:50<00:00,  2.31it/s]


Input: The overall probability of manager signing the termination letter is 30%. For managers who don't sign termination letters, the probability of large feet is 73%. For managers who sign termination letters, the probability of large feet is 25%. Is large feet more likely than small feet overall?
Predicted: no (confidence: 0.537)
Ground truth: yes


Evaluating: 100%|██████████| 100/100 [00:50<00:00,  1.98it/s]


Input: For those who are not yomx, the probability of xevu is 36%. For those who are yomx, the probability of xevu is 38%. For those who are yomx, would it be less likely to see xevu if the individual was not yomx?
Predicted: no (confidence: 0.558)
Ground truth: yes

Final Results:
accuracy: 0.4400
f1: 0.0968
precision: 0.4286
recall: 0.0545
brier_score: 0.3299
ece: 0.2653


In [ ]:
import torch
import numpy as np
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, roc_auc_score, log_loss
import nltk
from tqdm import tqdm
import matplotlib.pyplot as plt
from scipy.stats import entropy

def calculate_perplexity(model, tokenizer, input_text, device):
    """Calculate perplexity for a given input text."""
    try:
        inputs = tokenizer(input_text, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = model(**inputs, labels=inputs.input_ids)
            neg_log_likelihood = outputs.loss
            return torch.exp(neg_log_likelihood).item()
    except Exception as e:
        print(f"Perplexity calculation error: {str(e)}")
        return float('inf')

def calculate_kl_divergence(p_probs, q_probs):
    """Calculate KL divergence between two probability distributions."""
    p = np.array([p_probs, 1 - p_probs])
    q = np.array([q_probs, 1 - q_probs])
    return float(entropy(p, q))

def get_binary_prediction(model, tokenizer, input_text, device):
    """Generate a binary yes/no prediction with detailed confidence scores and metrics."""
    inputs = tokenizer(input_text, return_tensors="pt").to(device)

    with torch.no_grad():
        # Calculate perplexity first
        perplexity = calculate_perplexity(model, tokenizer, input_text, device)

        # Generate the response
        outputs = model.generate(
            inputs.input_ids,
            max_new_tokens=10,
            num_beams=2,
            do_sample=False,
            return_dict_in_generate=True,
            output_scores=True,
            pad_token_id=tokenizer.eos_token_id,
            early_stopping=True
        )

        # Get the predicted answer
        predicted_tokens = outputs.sequences[0][inputs.input_ids.shape[1]:]
        predicted_answer = tokenizer.decode(predicted_tokens, skip_special_tokens=True).strip().lower()

        # Calculate probabilities for yes/no
        last_token_logits = outputs.scores[-1][0].float()
        yes_token_ids = tokenizer.encode(" yes", add_special_tokens=False)
        no_token_ids = tokenizer.encode(" no", add_special_tokens=False)

        # Get average logits for yes/no tokens
        yes_logit = last_token_logits[yes_token_ids].mean()
        no_logit = last_token_logits[no_token_ids].mean()

        # Convert to probabilities using softmax
        logits = torch.tensor([yes_logit, no_logit], device=device)
        probs = torch.nn.functional.softmax(logits, dim=0)

        # Calculate entropy-based uncertainty
        probs_np = probs.cpu().numpy()
        uncertainty = entropy([probs_np[0], probs_np[1]], base=2) / np.log2(2)

        # Determine confidence level categories
        confidence = float(probs[0] if predicted_answer.startswith('yes') else probs[1])
        if confidence >= 0.9:
            confidence_level = "Very High"
        elif confidence >= 0.7:
            confidence_level = "High"
        elif confidence >= 0.5:
            confidence_level = "Moderate"
        else:
            confidence_level = "Low"

        return {
            'prediction': 'yes' if predicted_answer.startswith('yes') else 'no',
            'confidence': confidence,
            'confidence_level': confidence_level,
            'uncertainty': float(uncertainty),
            'yes_prob': float(probs[0]),
            'no_prob': float(probs[1]),
            'perplexity': perplexity,
            'logits': {
                'yes': float(yes_logit),
                'no': float(no_logit)
            }
        }

def calculate_binary_metrics(predictions, ground_truths):
    """Calculate comprehensive metrics for binary classification."""
    pred_labels = [1 if p['prediction'] == 'yes' else 0 for p in predictions]
    true_labels = [1 if gt == 'yes' else 0 for gt in ground_truths]
    confidences = [p['confidence'] for p in predictions]
    yes_probs = [p['yes_prob'] for p in predictions]

    # Basic classification metrics
    metrics = {
        'accuracy': accuracy_score(true_labels, pred_labels),
        'f1': f1_score(true_labels, pred_labels),
        'precision': precision_score(true_labels, pred_labels),
        'recall': recall_score(true_labels, pred_labels),
        'roc_auc': roc_auc_score(true_labels, yes_probs),
        'brier_score': np.mean([(c - t)**2 for c, t in zip(confidences, true_labels)]),
        'log_loss': log_loss(true_labels, yes_probs),
    }

    # Perplexity statistics
    perplexities = [p['perplexity'] for p in predictions]
    metrics['perplexity'] = {
        'mean': np.mean(perplexities),
        'std': np.std(perplexities),
        'min': np.min(perplexities),
        'max': np.max(perplexities)
    }

    # Confidence analysis
    confidence_levels = [p['confidence_level'] for p in predictions]
    metrics['confidence_distribution'] = {
        'Very High': confidence_levels.count('Very High') / len(confidence_levels),
        'High': confidence_levels.count('High') / len(confidence_levels),
        'Moderate': confidence_levels.count('Moderate') / len(confidence_levels),
        'Low': confidence_levels.count('Low') / len(confidence_levels)
    }

    # Calibration metrics
    n_bins = 10
    bins = np.linspace(0, 1, n_bins + 1)
    bin_indices = np.digitize(confidences, bins) - 1

    ece = 0.0  # Expected Calibration Error
    mce = 0.0  # Maximum Calibration Error

    for bin_idx in range(n_bins):
        mask = bin_indices == bin_idx
        if mask.any():
            bin_conf = np.mean([c for c, m in zip(confidences, mask) if m])
            bin_acc = np.mean([1 if p == t else 0
                             for p, t, m in zip(pred_labels, true_labels, mask) if m])
            bin_size = np.mean(mask)

            calibration_error = np.abs(bin_conf - bin_acc)
            ece += calibration_error * bin_size
            mce = max(mce, calibration_error)

    metrics['calibration'] = {
        'ece': float(ece),
        'mce': float(mce)
    }

    # Average metrics by confidence level
    for level in ['Very High', 'High', 'Moderate', 'Low']:
        level_indices = [i for i, l in enumerate(confidence_levels) if l == level]
        if level_indices:
            level_pred = [pred_labels[i] for i in level_indices]
            level_true = [true_labels[i] for i in level_indices]
            metrics[f'{level.lower()}_confidence_accuracy'] = accuracy_score(level_true, level_pred)

    return metrics

def evaluate_binary_dataset(model, tokenizer, dataset, device):
    """Evaluate the model on a dataset with comprehensive metrics."""
    all_predictions = []
    all_ground_truths = []

    for sample in tqdm(dataset, desc="Evaluating"):
        input_text = f"Given Info and Question: {sample['input']}\nAnswer:"
        ground_truth = str(sample['answer']).strip().lower()

        try:
            prediction = get_binary_prediction(model, tokenizer, input_text, device)
            all_predictions.append(prediction)
            all_ground_truths.append(ground_truth)

            # Print detailed sample results
            print(f"\nInput: {sample['input']}")
            print(f"Predicted: {prediction['prediction']}")
            print(f"Confidence: {prediction['confidence']:.3f} ({prediction['confidence_level']})")
            print(f"Perplexity: {prediction['perplexity']:.3f}")
            print(f"Uncertainty: {prediction['uncertainty']:.3f}")
            print(f"Ground truth: {ground_truth}")

        except Exception as e:
            print(f"Error processing sample: {str(e)}")

    # Calculate all metrics
    metrics = calculate_binary_metrics(all_predictions, all_ground_truths)


    return metrics

# Run evaluation
results = evaluate_binary_dataset(model, tokenizer, cladder_dataset, device)

# Print detailed results
print("\nDetailed Evaluation Results:")
for metric_name, value in results.items():
    if isinstance(value, dict):
        print(f"\n{metric_name}:")
        for sub_name, sub_value in value.items():
            print(f"  {sub_name}: {sub_value:.4f}")
    else:
        print(f"{metric_name}: {value:.4f}")

Evaluating:   1%|          | 1/100 [00:00<01:11,  1.39it/s]


Input: We know that male gender or in-state residency causes competitive department. male gender or in-state residency or competitive department causes admission acceptance. We observed the resident is in-state. Would the applicant gets rejected if male gender instead of non-male gender?
Predicted: yes
Confidence: 0.671 (Moderate)
Perplexity: 66.625
Uncertainty: 0.914
Ground truth: no


Evaluating:   2%|▏         | 2/100 [00:01<01:04,  1.52it/s]


Input: The overall probability of talent is 82%. For students who are not talented and rejected from elite institutions, the probability of being hard-working is 99%. For students who are not talented and accepted to elite institutions, the probability of being hard-working is 82%. For students who are talented and rejected from elite institutions, the probability of being hard-working is 96%. For students who are talented and accepted to elite institutions, the probability of being hard-working is 53%. If we look at students accepted to elite institutions, does the chance of being hard-working decrease when talent?
Predicted: no
Confidence: 0.576 (Moderate)
Perplexity: 6.801
Uncertainty: 0.983
Ground truth: yes


Evaluating:   3%|▎         | 3/100 [00:01<01:02,  1.54it/s]

...


Input: The overall probability of smoking mother is 15%. For infants with nonsmoking mothers, the probability of normal infant birth weight is 7%. For infants with smoking mothers, the probability of normal infant birth weight is 47%. Is normal infant birth weight more likely than low infant birth weight overall?
Predicted: no
Confidence: 0.953 (Very High)
Perplexity: 17.344
Uncertainty: 0.275
Ground truth: no


Evaluating:  99%|█████████▉| 99/100 [00:55<00:00,  1.91it/s]


Input: The overall probability of manager signing the termination letter is 30%. For managers who don't sign termination letters, the probability of large feet is 73%. For managers who sign termination letters, the probability of large feet is 25%. Is large feet more likely than small feet overall?
Predicted: no
Confidence: 0.537 (Moderate)
Perplexity: 21.672
Uncertainty: 0.996
Ground truth: yes


Evaluating: 100%|██████████| 100/100 [00:55<00:00,  1.79it/s]


Input: For those who are not yomx, the probability of xevu is 36%. For those who are yomx, the probability of xevu is 38%. For those who are yomx, would it be less likely to see xevu if the individual was not yomx?
Predicted: no
Confidence: 0.558 (Moderate)
Perplexity: 15.703
Uncertainty: 0.990
Ground truth: yes

Detailed Evaluation Results:
accuracy: 0.4400
f1: 0.0968
precision: 0.4286
recall: 0.0545
roc_auc: 0.4832
brier_score: 0.3299
log_loss: 1.3498

perplexity:
  mean: 22.3496
  std: 17.5182
  min: 3.8418
  max: 83.2500

confidence_distribution:
  Very High: 0.3500
  High: 0.2500
  Moderate: 0.1800
  Low: 0.2200

calibration:
  ece: 0.3665
  mce: 0.5921
very high_confidence_accuracy: 0.3714
high_confidence_accuracy: 0.4800
moderate_confidence_accuracy: 0.4444
low_confidence_accuracy: 0.5000
